In [2]:
file_name = '../02_combine_deepdao_aragon_and_messari/messari_deepdao_aragon_organizations.csv'

In [3]:
import pandas as pd
df = pd.read_csv(file_name)
df.head()

,deepdaoDaoId,deepdaoOrganizationId,name,platformTitle,platformId,website,mostRecentProposalDate,aum
0,17ef523a-6588-4d0a-a0d6-12dad7b88075,11eb10d7-6493-4cfc-8255-cb0f2338fa67,Balancer,Safe/Snapshot,12,https://snapshot.org/#/balancer.eth,2023-06-15T22:10:19.000Z,2.333428e+07
1,58bf8009-2b6c-4dee-9d00-df584296a391,1323d31d-bd0c-4605-ba55-573dd7129324,Mango DAO,Realms,14,https://www.mango.markets/,2023-06-20T06:50:30.000Z,6.756388e+07
2,43c2285d-7d6c-498e-9630-f91df72ec092,17de4869-1c7f-4c30-9f25-acb71b196cb1,Aura Finance,Snapshot,11,NaN,2023-06-16T22:58:25.000Z,3.571961e+07
3,1ce1d6da-483a-4a01-8c01-433484f6ddbc,18abd2d8-ff90-40cd-a889-4cc2fed78364,Aragon One onchain: Aragon governance,Aragon,4,NaN,NaN,NaN
4,5bd9212c-8927-4951-a1e1-e939a1c5fb7e,18abd2d8-ff90-40cd-a889-4cc2fed78364,Aragon Trust,Aragon,4,https://mainnet.aragon.org/#/trust.aragonid.eth,2023-05-12T07:14:35.000Z,NaN


In [4]:
# get pages with platform title is Snapshot
df = df[df['platformTitle'] == 'Snapshot']
df.shape

(120, 8)

In [5]:
df.head()

,deepdaoDaoId,deepdaoOrganizationId,name,platformTitle,platformId,website,mostRecentProposalDate,aum
2,43c2285d-7d6c-498e-9630-f91df72ec092,17de4869-1c7f-4c30-9f25-acb71b196cb1,Aura Finance,Snapshot,11,NaN,2023-06-16T22:58:25.000Z,3.571961e+07
7,9c1c28fe-d487-4c15-8d8b-0cfd86607ea0,18abd2d8-ff90-40cd-a889-4cc2fed78364,Aragon Network DAO,Snapshot,11,https://snapshot.org/#/aclay.eth,2022-06-14T11:56:50.000Z,NaN
8,ab82af48-3631-4fc8-993e-320c2ebba5fd,18abd2d8-ff90-40cd-a889-4cc2fed78364,Aragon,Snapshot,11,https://snapshot.org/#/aragon,2021-04-07T21:07:11.000Z,NaN
10,bdfd22a5-7055-417d-9e75-112838d7227e,1f639420-2f49-44f0-b8ed-9f3c811e7205,frami y otras hierbas,Snapshot,11,https://snapshot.org/#/frami.eth,2022-09-23T17:34:22.000Z,NaN
12,f41dad8c-b3c4-4884-864b-eadb7f1b85d8,219ab5ef-9345-448b-bbf2-786a33e4edd8,Aavegotchi,Snapshot,11,https://snapshot.org/#/aavegotchi.eth,2023-06-19T08:46:45.000Z,NaN


In [6]:
# epoch from a year ago
import datetime
import time
year_ago = datetime.datetime.now() - datetime.timedelta(days=365)
YEAR_AGO_EPOCH = int(time.mktime(year_ago.timetuple()))
YEAR_AGO_EPOCH

1655981391

In [7]:
def generate_query(space, first, skip):
    base = """
        query Proposals {
         proposals(where: {space: "%s", created_gte: %s}, first: %s, skip: %s, orderDirection: desc, orderBy: "created") {
            id
            ipfs
            author
            created
            network
            symbol
            type
            strategies {
              network
              params
            }
            validation {
              params
            }
            plugins
            title
            # body
            discussion
            choices
            start
            end
            quorum
            privacy
            snapshot
            state
            link
            app
            scores
            scores_by_strategy
            scores_state
            scores_total
            scores_updated
            votes
            flagged
          }
        }
    """
    return base % (space, YEAR_AGO_EPOCH, first, skip)

In [8]:
df.iloc[0].website

nan

In [9]:
# get the part before the final slash and put it into a snapshotId column, handle nulls
df['snapshotId'] = df['website'].apply(lambda x: x.split('/')[-1] if type(x) == str else None)
df.head()

,deepdaoDaoId,deepdaoOrganizationId,name,platformTitle,platformId,website,mostRecentProposalDate,aum,snapshotId
2,43c2285d-7d6c-498e-9630-f91df72ec092,17de4869-1c7f-4c30-9f25-acb71b196cb1,Aura Finance,Snapshot,11,NaN,2023-06-16T22:58:25.000Z,3.571961e+07,None
7,9c1c28fe-d487-4c15-8d8b-0cfd86607ea0,18abd2d8-ff90-40cd-a889-4cc2fed78364,Aragon Network DAO,Snapshot,11,https://snapshot.org/#/aclay.eth,2022-06-14T11:56:50.000Z,NaN,aclay.eth
8,ab82af48-3631-4fc8-993e-320c2ebba5fd,18abd2d8-ff90-40cd-a889-4cc2fed78364,Aragon,Snapshot,11,https://snapshot.org/#/aragon,2021-04-07T21:07:11.000Z,NaN,aragon
10,bdfd22a5-7055-417d-9e75-112838d7227e,1f639420-2f49-44f0-b8ed-9f3c811e7205,frami y otras hierbas,Snapshot,11,https://snapshot.org/#/frami.eth,2022-09-23T17:34:22.000Z,NaN,frami.eth
12,f41dad8c-b3c4-4884-864b-eadb7f1b85d8,219ab5ef-9345-448b-bbf2-786a33e4edd8,Aavegotchi,Snapshot,11,https://snapshot.org/#/aavegotchi.eth,2023-06-19T08:46:45.000Z,NaN,aavegotchi.eth


In [10]:
# keep only rows with a snapshotId
df = df[df['snapshotId'].notnull()]
# reindex
df = df.reset_index(drop=True)
df.shape

(119, 9)

In [11]:
import requests
import json
import pandas as pd

URL = "https://hub.snapshot.org/graphql"


def get_all_proposals(snapshotId):
    all_proposals = []

    has_next_page = True
    skip = 0
    while has_next_page:
        print('.')

        query = generate_query(snapshotId, 1000, skip)
        r = requests.post(URL, json={'query': query})
        data = json.loads(r.text)

        proposals = data['data']['proposals']
        all_proposals += proposals

        if len(proposals) < 1000:
            has_next_page = False
        skip += 1000

    all_proposals_df = pd.DataFrame(all_proposals)

    # add snapshotId to the df
    all_proposals_df['snapshotId'] = snapshotId
    return all_proposals_df


sid = 'magicappstore.eth'

test_proposals = get_all_proposals(sid)
test_proposals.shape

.


(91, 29)

In [12]:
test_proposals.tail()

,id,ipfs,author,created,network,symbol,type,strategies,validation,plugins,...,link,app,scores,scores_by_strategy,scores_state,scores_total,scores_updated,votes,flagged,snapshotId
86,0x78af69810183e97bccb9e04133d715d956d86c4e7349...,bafkreieurhlygbw2p6pamfivn74ihxjumngzchl52zmd5...,0x17A4382bd5cAB0e3D452b9e8BF07C96BC5Ec817A,1683560851,56,Magic Store Vote,single-choice,"[{'network': '56', 'params': {'url': 'https://...",{'params': {}},{},...,https://snapshot.org/#/magicappstore.eth/propo...,cross-the-ages,"[1107, 55]","[[1107], [55]]",final,1162,1684905010,1162,False,magicappstore.eth
87,0x0a3fd11d0d2e360a4f9ba95fc45e13f42541209d0017...,bafkreidkhhynwutbpvcxl26e25u22csjqsamcwqjzyajr...,0x17A4382bd5cAB0e3D452b9e8BF07C96BC5Ec817A,1683560568,56,Magic Store Vote,single-choice,"[{'network': '56', 'params': {'url': 'https://...",{'params': {}},{},...,https://snapshot.org/#/magicappstore.eth/propo...,metawear-fashiondome,"[1049, 86]","[[1049], [86]]",final,1135,1684904723,1135,False,magicappstore.eth
88,0xca20b7331789416525ef422dace3ac621baedd672073...,bafkreie53m7vfz3lfiyk45nyqemz5prro765bnioa4vbr...,0x17A4382bd5cAB0e3D452b9e8BF07C96BC5Ec817A,1683559583,56,Magic Store Vote,single-choice,"[{'network': '56', 'params': {'url': 'https://...",{'params': {}},{},...,https://snapshot.org/#/magicappstore.eth/propo...,kryxivia-game,"[1076, 61]","[[1076], [61]]",final,1137,1684904989,1137,False,magicappstore.eth
89,0xc1b651e953a24b93ae1c48917f3fba2e14d8ea4dadf0...,bafkreiawdormlimrgbuy4syzz6tz6azob6ay2ynxbh2qd...,0x17A4382bd5cAB0e3D452b9e8BF07C96BC5Ec817A,1683546940,56,Magic Store Vote,single-choice,"[{'network': '56', 'params': {'url': 'https://...",{'params': {}},{},...,https://snapshot.org/#/magicappstore.eth/propo...,war-legends,"[1179, 62]","[[1179], [62]]",final,1241,1684904846,1241,False,magicappstore.eth
90,0x29a572ca57e9648c5d4d0715d895724ae5782d10276b...,bafkreidn4e422ccn4s2f6hifgf4kranxvrea55ubfhkzq...,0x17A4382bd5cAB0e3D452b9e8BF07C96BC5Ec817A,1683050996,56,Magic Store Vote,single-choice,"[{'network': '56', 'params': {'url': 'https://...",{'params': {}},{},...,https://snapshot.org/#/magicappstore.eth/propo...,validation-simulator,"[7047, 144]","[[7047], [144]]",pending,7191,1687516720,7191,False,magicappstore.eth


In [13]:
# get the current epoch and make a directory
import time
import datetime
now = datetime.datetime.now()
EPOCH = int(time.mktime(now.timetuple()))
EPOCH
# make a directory
import os
directory = f'./{EPOCH}'
if not os.path.exists(directory):
    os.makedirs(directory)

In [14]:
df.shape

(119, 9)

In [15]:
LIMIT = 30
# DIRECTORY = '1687429408'

In [16]:
for row in df[:LIMIT].itertuples():
    sid = row.snapshotId  # this is the key to query the api
    dao_id = row.deepdaoDaoId    # this is the foreign key we use to link back to the DeepDAO data

    if os.path.exists(f'{directory}/{dao_id}.csv'):
        print(f'{directory}/{dao_id}.csv already exists, skipping')
        continue

    proposals = get_all_proposals(sid)

    # save w daoId
    proposals.to_csv(f'{directory}/{dao_id}.csv', index=False)
    
    # add random wait
    import random
    import time
    wait_time = random.randint(1, 10)
    print(f'waiting {wait_time} seconds')
    time.sleep(wait_time)


.
waiting 1 seconds
.
waiting 5 seconds
.
waiting 5 seconds
.
waiting 5 seconds
.
waiting 1 seconds
.
waiting 1 seconds
.
waiting 1 seconds
.
waiting 6 seconds
.
waiting 3 seconds
.
waiting 7 seconds
.
waiting 5 seconds
.
waiting 3 seconds
.
waiting 5 seconds
.
waiting 2 seconds
.
waiting 8 seconds
.
waiting 6 seconds
.
waiting 2 seconds
.
waiting 6 seconds


KeyboardInterrupt: 